In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!python --version

Python 3.10.12


In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 61.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 38.3 MB/s eta 0:00:00


In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW, TrainingArguments, Trainer
import numpy as np

In [ ]:
# Load BioLinkBERT tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('michiyasunaga/BioLinkBERT-base')
model = AutoModelForSequenceClassification.from_pretrained('michiyasunaga/BioLinkBERT-base', num_labels=2)

# Other Models
# allenai/scibert_scivocab_uncased
# domenicrosati/ClinicalTrialBioBert-NLI4CT
# medicalai/ClinicalBERT

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at michiyasunaga/BioLinkBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
!pwd

/content


In [ ]:
cd drive/MyDrive/NLI4CT

/content/drive/MyDrive/NLI4CT


In [ ]:
import os
import json

data = json.load(open('./train.json'))
CT_files = os.listdir("./CT json")
# CT_files.remove(".DS_Store")

CT_files_data = {file[:-5]:json.load(open(f"./CT json/{file}")) for file in CT_files}

data_expanded = []
for _id, value in data.items():
    temp = {}
    temp["id"] = _id
    p_nctid = value["Primary_id"]
    s_nctid = value.get("Secondary_id")
    section_id = value["Section_id"]
    statement = value["Statement"]
    primary_evidence = CT_files_data[p_nctid][section_id]
    temp["statement"] = statement
    temp["primary_evidence"] = primary_evidence
    temp["label"] = value["Label"]

    if s_nctid is not None:
        secondary_evidence = CT_files_data[s_nctid][section_id]
        temp["secondary_evidence"] = secondary_evidence

    data_expanded.append(temp)

data_expanded[0]

{'id': '5bc844fc-e852-4270-bfaf-36ea9eface3d',
 'statement': 'All the primary trial participants do not receive any oral capecitabine, oral lapatinib ditosylate or cixutumumab IV, in conrast all the secondary trial subjects receive these.',
 'primary_evidence': ['INTERVENTION 1: ',
  '  Diagnostic (FLT PET)',
  '  Patients with early stage, ER positive primary breast cancer undergo FLT PET scan at baseline and 1-6 weeks after the start of standard endocrine treatment. The surgery follows 1-7 days after the second FLT PET scan.',
  '  Tracer used in the FLT PET (positron emission tomography) scanning procedure: [F18] fluorothymidine.',
  '  Positron Emission Tomography: Undergo FLT PET',
  '  Laboratory Biomarker Analysis: Correlative studies - Ki67 staining of the tumor tissue in the biopsy and surgical specimen.'],
 'label': 'Contradiction',
 'secondary_evidence': ['INTERVENTION 1: ',
  '  Arm A',
  '  Patients receive oral capecitabine twice daily on days 1-14 and oral lapatinib dito

In [ ]:
samples = []
for sample in data_expanded:
    primary_evidence = "".join(sample['primary_evidence'])
    sentence = f"Primary trial evidence are \n {primary_evidence}"
    secondary_evidence = sample.get("secondary_evidence")
    if secondary_evidence:
        secondary_evidence = "".join(sample['secondary_evidence'])
        sentence = f"{sentence}\n Secondary trial evidence are \n{secondary_evidence}"
    temp = {"id": sample['id'], "clinical_trial":sentence, "hypothesis":sample['statement'], "label":sample['label']}
    samples.append(temp)

samples[80]

{'id': 'b6ac985d-87ae-4e0f-83e1-38033c1db5cc',
 'clinical_trial': 'Primary trial evidence are \n Inclusion Criteria:  Pre-menopausal women aged 18 years or over with histologically/cytologically-confirmed oestrogen receptor positive (ER +ve) breast cancer  World Health Organization (WHO) performance status of 0, 1, or 2  Provided written informed consentExclusion Criteria:  Treatment with tamoxifen or other hormonal therapies as early breast cancer (EBC) adjuvant in the previous 24 weeks  Received radiotherapy within the past 4 weeks  History of systemic malignancy other than breast cancer within the previous 3 years  Estimated survival less than 24 weeks\n Secondary trial evidence are \nInclusion Criteria:  Recurrent or residual metastatic breast carcinoma  Zubrod performance status less than 2  18-60 years old  Related donor human leukocyte antigen (HLA)-compatible for allogeneic transplantation or unrelated HLA-compatible donor.  No major organ dysfunction or active infectionExclusi

In [ ]:
premises = []
hypotheses = []
label = []
for sample in samples:
  premises.append(sample['clinical_trial'])
  hypotheses.append(sample['hypothesis'])
  label.append(1 if sample['label'] == "Entailment" else 0)

In [ ]:
# Tokenize input data and create input tensors
input_ids_list = []
attention_masks_list = []

# Tokenize input data and pad to the same length
max_length = 512

for premise, hypothesis in zip(premises, hypotheses):
    encoding = tokenizer(premise, hypothesis, padding='max_length', truncation=True, return_tensors="pt", max_length=max_length)
    input_ids_list.append(encoding["input_ids"])
    attention_masks_list.append(encoding["attention_mask"])

input_ids = torch.stack(input_ids_list)
attention_masks = torch.stack(attention_masks_list)
labels = torch.tensor(label)

In [ ]:
# Define custom model (modify this for your specific task)
class BERTModel(torch.nn.Module):
    def __init__(self, pretrained_model, num_labels):
        super().__init__()
        self.pretrained_model = pretrained_model
        self.classifier = torch.nn.Linear(pretrained_model.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask):
        outputs = self.pretrained_model(input_ids=input_ids, attention_mask=attention_mask)
        # print(outputs)
        # pooled_output = outputs.pooler_output  # Use the pooled_output as the final representation
        logits = self.classifier(outputs.logits)
        return logits

bert_model = BERTModel(model, num_labels=2)

In [ ]:
from sklearn.model_selection import train_test_split

# Split data into train and validation sets
train_inputs, val_inputs, train_masks, val_masks, train_labels, val_labels = train_test_split(
    input_ids, attention_masks, labels, test_size=0.15, random_state=42
)

# Create DataLoader objects for training and validation
train_dataset = TensorDataset(train_inputs, train_masks, train_labels)
val_dataset = TensorDataset(val_inputs, val_masks, val_labels)

train_loader = DataLoader(train_dataset, batch_size=8)
val_loader = DataLoader(val_dataset, batch_size=8)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28895, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
def evaluate(data_loader):
  model.eval()
  total_val_loss = 0
  true_labels = []
  predicted_labels = []

  from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

  with torch.no_grad():
      for batch in data_loader:
          input_ids_batch, attention_mask_batch, labels_batch = batch

          input_ids_batch = input_ids_batch.squeeze(1)
          attention_mask_batch = attention_mask_batch.squeeze(1)
          input_ids_batch = input_ids_batch.to(device)
          attention_mask_batch = attention_mask_batch.to(device)
          labels_batch = labels_batch.to(device)

          # Forward pass
          outputs = model(input_ids_batch, attention_mask=attention_mask_batch)
          logits = outputs.logits  # Logits for classification

          # Pass logits to the loss function
          loss = loss_fn(logits, labels_batch)
          total_val_loss += loss.item()

          # Calculate predicted labels
          predicted_batch = torch.argmax(logits, dim=1)

          true_labels.extend(labels_batch.cpu().numpy())  # Convert to NumPy array
          predicted_labels.extend(predicted_batch.cpu().numpy())  # Convert to NumPy array

  # Calculate accuracy
  accuracy = accuracy_score(true_labels, predicted_labels)
  average_val_loss = total_val_loss / len(val_loader)

  # Calculate precision, recall, and F1 score
  precision = precision_score(true_labels, predicted_labels, average='weighted')
  recall = recall_score(true_labels, predicted_labels, average='weighted')
  f1 = f1_score(true_labels, predicted_labels, average='weighted')

  print(f"Loss: {average_val_loss}", end=" ... ")
  print(f"Accuracy: {accuracy}", end=" ... ")
  print(f"Precision: {precision}", end=" ... ")
  print(f"Recall: {recall}", end=" ... ")
  print(f"F1 Score: {f1}")
  return average_val_loss, accuracy, precision, recall, f1


In [ ]:
# Define optimizer and loss function
optimizer = AdamW(model.parameters(), lr=1e-7)
loss_fn = torch.nn.CrossEntropyLoss()

train_acc = []
val_acc = []
train_loss = []
val_loss = []
train_prec = []
val_prec = []
train_rec = []
val_rec = []
train_f1 = []
val_f1 = []

# Training loop
for epoch in range(50):
    print(f"\n######### {epoch} ###########")
    model.train()
    print("Training Metrics")
    l, a, p, r, f = evaluate(train_loader)
    train_acc.append(a)
    train_loss.append(l)
    train_prec.append(p)
    train_rec.append(r)
    train_f1.append(f)
    print("Validation Metrics")
    l, a, p, r, f = evaluate(val_loader)
    val_acc.append(a)
    val_loss.append(l)
    val_prec.append(p)
    val_rec.append(r)
    val_f1.append(f)
    for batch in train_loader:
        input_ids_batch, attention_mask_batch, labels_batch = batch
        input_ids_batch = input_ids_batch.squeeze(1)
        attention_mask_batch = attention_mask_batch.squeeze(1)
        input_ids_batch = input_ids_batch.to(device)
        attention_mask_batch = attention_mask_batch.to(device)
        labels_batch = labels_batch.to(device)
        # print(input_ids_batch.shape)
        # print(attention_mask_batch.shape)
        optimizer.zero_grad()
        logits = model(input_ids=input_ids_batch, attention_mask=attention_mask_batch).logits
        loss = loss_fn(logits, labels_batch)
        loss.backward()
        optimizer.step()
    torch.save(model.state_dict(), f'bioLinkBert{epoch}.pth')

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(



######### 0 ###########
Training Metrics
Loss: 3.9915558006614447 ... Accuracy: 0.4955017301038062 ... Precision: 0.492390821868768 ... Recall: 0.4955017301038062 ... F1 Score: 0.46598520205149063
Validation Metrics
Loss: 0.7051220573484898 ... Accuracy: 0.5098039215686274 ... Precision: 0.5262459150326798 ... Recall: 0.5098039215686274 ... F1 Score: 0.4767650650003592

######### 1 ###########
Training Metrics
Loss: 3.9640215430408716 ... Accuracy: 0.4996539792387543 ... Precision: 0.4985619967156685 ... Recall: 0.4996539792387543 ... F1 Score: 0.4871571605002031
Validation Metrics
Loss: 0.6991189736872911 ... Accuracy: 0.5254901960784314 ... Precision: 0.5348951655929733 ... Recall: 0.5254901960784314 ... F1 Score: 0.5146283799652414

######### 2 ###########
Training Metrics
Loss: 3.9513072166591883 ... Accuracy: 0.5031141868512111 ... Precision: 0.5025409807416729 ... Recall: 0.5031141868512111 ... F1 Score: 0.4962643390838292
Validation Metrics
Loss: 0.6969400644302368 ... Accuracy